<a href="https://colab.research.google.com/github/biplobsd/Google-Colab-CloudTorrent/blob/master/torrentTOmega_gdrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center><a href="https://t.me/torrentToGM"><img src='https://i.imgur.com/CLg6blO.png' height="70" alt="Telegram Group"/></a><a href="https://github.com/biplobsd/Google-Colab-CloudTorrent"><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/e/eb/Ei-sc-github.svg/768px-Ei-sc-github.svg.png' height="80" alt="Github"/></a><img src="https://hitcounter.pythonanywhere.com/count/tag.svg?url=https%3A%2F%2Fgithub.com%2Fbiplobsd%2FGoogle-Colab-CloudTorrent" alt="Hits" align="right"></center>

___Preview___ <img src="https://media.giphy.com/media/3kIHVWVw2rDIfJyiN1/giphy.gif" height=15px>
![preview](https://raw.githubusercontent.com/biplobsd/Google-Colab-CloudTorrent/master/src/preview.gif)

___aria2 Rpc Config Setup___ <img src="https://media.giphy.com/media/3kIHVWVw2rDIfJyiN1/giphy.gif" height=15px>
![preview](https://raw.githubusercontent.com/biplobsd/Google-Colab-CloudTorrent/master/src/aria2RpcConfigSetup.gif)

# <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png" height=20px> __Mount Gdrive!__

In [0]:
#Mount your Gdrive! 
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# <img src='https://i.imgur.com/DOoxSuO.png' height="38" alt="Peerflix-server" />



In [0]:
#@markdown <h3>⬅️ Click Here to START server</h3>
#@markdown <br><center><img src='https://i.imgur.com/DOoxSuO.png' height="50" alt="netdata"/></center>
#@markdown <center><h3>SimpleTorrent is a a self-hosted remote torrent client.</h3></center><br>
#@markdown <center><h5>Auto upload to gdrive after task completed. if gdrive is mounted.</h5></center><br>

import os
import sys
import subprocess
import shlex
import time
import string
import urllib.request
import json
from IPython.display import HTML, clear_output

# script version
Version = '1.0.9'

#####################################
USE_FREE_TOKEN = True  # @param {type:"boolean"}
Auto_UP_Gdrive = False  # @param {type:"boolean"}
TOKEN = ""  # @param {type:"string"}
PORT = 4444

if not os.path.exists("/root/.ipython/ttmg.py"):
    shellCmd = "wget -qq https://raw.githubusercontent.com/biplobsd/" \
                "Google-Colab-CloudTorrent/master/res/ttmg.py" \
                " -O /root/.ipython/ttmg.py"
    subprocess.run(shlex.split(shellCmd))

from ttmg import (
    updateCheck,
    runSh,
    nameport,
    installNgrok,
    findProcess,
    displayUrl,
    loadingAn
)

if Auto_UP_Gdrive:
  dcmd = "wget -qq https://raw.githubusercontent.com/biplobsd/" \
            "Google-Colab-CloudTorrent/master/res/scripts/doneCMD.sh" \
            " -O /usr/local/bin/doneCMD.sh &"
  runSh(dcmd)
  runSh('chmod 0755 /usr/local/bin/doneCMD.sh')
else:
  runSh('rm -rf /usr/local/bin/doneCMD.sh')
  
configPath = "/content/cloud-torrent.json"
configsdata = r"""
{
  "AutoStart": true,
  "EngineDebug": false,
  "MuteEngineLog": true,
  "ObfsPreferred": true,
  "ObfsRequirePreferred": false,
  "DisableTrackers": false,
  "DisableIPv6": false,
  "DownloadDirectory": "/content/downloads/",
  "WatchDirectory": "/content/torrents/",
  "EnableUpload": true,
  "EnableSeeding": false,
  "IncomingPort": 50007,
  "DoneCmd": "/usr/local/bin/doneCMD.sh",
  "SeedRatio": 1.5,
  "UploadRate": "High",
  "DownloadRate": "Unlimited",
  "TrackerListURL": "https://raw.githubusercontent.com/ngosang/trackerslist/master/trackers_best.txt",
  "AlwaysAddTrackers": true,
  "ProxyURL": ""
}
""".split("\n")
open(configPath, 'w').close()
with open(configPath, "a+") as configFile:
    for line in configsdata:
        configFile.write(line + "\n")
#####################################


def exx(echo=None):
  if echo:
    print(echo)
  !kill -9 -1 &

def ngrok_config(token, region, port, configPath="/root/.ngrok2/ngrok.yml"):
  data = """
  authtoken: {}
  region: {}
  update: false
  update_channel: stable
  web_addr: localhost:{}
  tunnels:
    simple-torrent:
      addr: 4444
      proto: http
      inspect: false
    peerflix-server:
      addr: 4445
      proto: http
      inspect: false
  """.format(token, region, port).split('\n')
  try:
    os.mkdir('/root/.ngrok2/')
  except:
    pass
  open(configPath, 'w').close()
  with open(configPath, "a+") as configFile:
    for line in data:
      configFile.write(line + "\n")
  return True

def startWebUi(data, dport, nServer, region, btc, configPath):
    tokens, WEB_PORT = data
    from json import loads

    if tokens == "Invalid Token":
        print(tokens)
        exx()

    installNgrok()
    clear_output(wait=True)
    loadingAn(name="lds")
    print("Starting ngrok ...")
    ngrok_config(tokens, region, dport, configPath)
    shellCmd = f"ngrok start --config {configPath} --all &"
    runSh(shellCmd, shell=True)
    time.sleep(7)
    try:
        host = urllib.request.urlopen(f"http://localhost:{dport}/api/tunnels")
        host = json.loads(host.read())['tunnels']
        for h in host:
          if h['name'] == nServer:
            host = h['public_url'][8:]
            break
    except:
        print("ngrok Token is in used!. Try another token ...")
        time.sleep(2)
        return True
    
    data = {
        "url": f"http://{host}",
        "port": WEB_PORT,
        "token": tokens,
    }
    displayUrl(data, btc)
    return data

if updateCheck("Checking updates ...", Version):  # VERSION CHECKING ...
    exx()
clear_output()

# Simple Torrent installing ...
loadingAn()
if os.path.isfile("/usr/local/bin/cloud-torrent") is False:
    dcmd = "wget -qq https://raw.githubusercontent.com/biplobsd/" \
            "Google-Colab-CloudTorrent/master/res/scripts/" \
            "simpleCloudInstaller.sh -O ./simpleCloudInstaller.sh"
    runSh(dcmd)
    runSh('bash ./simpleCloudInstaller.sh')
    runSh('rm -rf ./simpleCloudInstaller.sh')
    try:
        os.mkdir('downloads')
        os.mkdir('torrents')
    except FileExistsError:
        pass


#Opening cloud-torrent in background
if not findProcess("cloud-torrent"):
  cmdC = f'cloud-torrent --port {PORT} ' \
      '-t "SimpleTorrent" -c /content/cloud-torrent.json &'
  for run in range(10):    
    runSh(cmdC,shell=True)
    time.sleep(3)
    try:
      urllib.request.urlopen(f"http://localhost:{PORT}")
      break
    except:
      print("Error: Simple-Torrent not starting. Retrying ...")
      runSh('pkill cloud-torrent')

# START_SERVER
dport = 4040
nServer = 'simple-torrent'
btc = 'b'
configPath = "/root/.ngrok2/ngrok1.yml"
region = 'us'

try:
  host = urllib.request.urlopen(f"http://localhost:{dport}/api/tunnels")
  host = json.loads(host.read())['tunnels']
  succ = False
  for h in host:
    if h['name'] == nServer:
      host = h['public_url'][8:]
      data = {
        "url": f"http://{host}",
        "port": PORT,
      }
      displayUrl(data, btc)
      succ = True
      break
  if not succ:
    raise Exception('Not found tunnels')
except:
  for run in range(10):
    clear_output()
    loadingAn(name='lds')
    try:
      dati = startWebUi(
          nameport(TOKEN, USE_FREE_TOKEN, PORT),
          dport,
          nServer,
          region,
          btc,
          configPath
          )
      if dati == True:
        continue
      urllib.request.urlopen(dati['url'])
      break
    except:
      clear_output()
      print(f"[{run}] Error: Ngrok not starting. Retrying ...")

# <img src='https://camo.githubusercontent.com/95b0ba2a237ee846b3bb8f45954124eb9cba4ebc/68747470733a2f2f63646e2e6a7364656c6976722e6e65742f67682f617361706163682f70656572666c69782d736572766572406d61737465722f6170702f696d616765732f6c6f676f2e737667' height="25" alt="Peerflix-server" /> __PEERFLIX-SERVER__

In [0]:
#@markdown <h3>⬅️ Click Here to START server</h3>
#@markdown <br><center><img src='https://camo.githubusercontent.com/95b0ba2a237ee846b3bb8f45954124eb9cba4ebc/68747470733a2f2f63646e2e6a7364656c6976722e6e65742f67682f617361706163682f70656572666c69782d736572766572406d61737465722f6170702f696d616765732f6c6f676f2e737667' height="200" alt="netdata"/></center>
#@markdown <center><h3>PEERFLIX-SERVER<br />Streaming torrent client for node.js with web ui.</h3></center><br>
import sys, shlex, subprocess, string, os, urllib.request, json, time; from IPython.display import HTML, clear_output

Version = '1.0.9'

#####################################
USE_FREE_TOKEN = True  # @param {type:"boolean"}
TOKEN = ""  # @param {type:"string"}
PORT = 4445

if not os.path.exists("/root/.ipython/ttmg.py"):
    shellCmd = "wget -qq https://raw.githubusercontent.com/biplobsd/" \
                "Google-Colab-CloudTorrent/master/res/ttmg.py \
                    -O /root/.ipython/ttmg.py"
    subprocess.run(shlex.split(shellCmd))

from ttmg import (
    updateCheck,
    runSh,
    nameport,
    installNgrok,
    findProcess,
    displayUrl,
    loadingAn,
)

def exx():
  !kill -9 -1 &

def ngrok_config(token, region, port, configPath="/root/.ngrok2/ngrok.yml"):
  data = """
  authtoken: {}
  region: {}
  update: false
  update_channel: stable
  web_addr: localhost:{}
  tunnels:
    simple-torrent:
      addr: 4444
      proto: http
      inspect: false
    peerflix-server:
      addr: 4445
      proto: http
      inspect: false
  """.format(token, region, port).split('\n')
  try:
    os.mkdir('/root/.ngrok2/')
  except:
    pass
  open(configPath, 'w').close()
  with open(configPath, "a+") as configFile:
    for line in data:
      configFile.write(line + "\n")
  return True

def startWebUi(data, dport, nServer, region, btc, configPath):
    tokens, WEB_PORT = data
    from json import loads

    if tokens == "Invalid Token":
        print(tokens)
        exx()

    installNgrok()
    clear_output(wait=True)
    loadingAn(name="lds")
    print("Starting ngrok ...")
    ngrok_config(tokens, region, dport, configPath)
    shellCmd = f"ngrok start --config {configPath} --all &"
    runSh(shellCmd, shell=True)
    time.sleep(7)
    try:
        host = urllib.request.urlopen(f"http://localhost:{dport}/api/tunnels")
        host = json.loads(host.read())['tunnels']
        for h in host:
          if h['name'] == nServer:
            host = h['public_url'][8:]
            break
    except:
        print("ngrok Token is in used!. Try another token ...")
        time.sleep(2)
        return True
    
    data = {
        "url": f"http://{host}",
        "port": WEB_PORT,
        "token": tokens,
    }
    displayUrl(data, btc)
    return data

#####################################

loadingAn()
# SAVE THE CONFIG
configPath = r"/root/.config/peerflix-server/config.json"
if not os.path.isfile(configPath):
  #CREATE THE CONFIG
  data = r"""
  { 
    "path": "/content/peerflix",
    "connections": 400
  }
  """.split("\n")
  os.mkdir('/root/.config/peerflix-server/')
  open(configPath, 'w').close()
  with open(configPath, "a+") as configFile:
    for line in data:
      configFile.write(line + "\n")

if not os.path.isfile("/tools/node/bin/peerflix-server"):
  runSh('npm install -g peerflix-server')

#Opening peerflix-server in background
if not os.path.exists("/content/peerflix"):
  os.mkdir('peerflix')

#Opening Peerflix-server in background
if not findProcess("peerflix-server"):
  cmdC = f'PORT={PORT} peerflix-server &'
  for run in range(10):    
    runSh(cmdC,shell=True)
    time.sleep(3)
    try:
      urllib.request.urlopen(f"http://localhost:{PORT}")
      break
    except:
      print("Error: Peerflix-server not starting. Retrying ...")
      runSh('pkill peerflix-server')

# START_SERVER
dport = 4040
nServer = 'peerflix-server' 
configPath = "/root/.ngrok2/ngrok1.yml"
region = 'us'
btc = 'r'

try:
  host = urllib.request.urlopen(f"http://localhost:{dport}/api/tunnels")
  host = json.loads(host.read())['tunnels']
  succ = False
  for h in host:
    if h['name'] == nServer:
      host = h['public_url'][8:]
      data = {
        "url": f"http://{host}",
        "port": PORT,
      }
      displayUrl(data, btc)
      succ = True
      break
  if not succ:
    raise Exception('Not found tunnels')
except:
  for run in range(10):
    clear_output()
    loadingAn(name='lds')
    try:
      dati = startWebUi(
          nameport(TOKEN, USE_FREE_TOKEN, PORT),
          dport,
          nServer,
          region,
          btc,
          configPath
          )
      if dati == True:
        continue
      urllib.request.urlopen(dati['url'])
      break
    except:
      clear_output()
      print(f"[{run}] Error: Ngrok not starting. Retrying ...")


In [0]:

#@markdown <br><center><img src='https://i.imgur.com/YWSbyoZ.png' height="50" alt="netdata"/></center>
#@markdown <br><center><h1>CLEAN DOWNLODAS FOLDER</center>
#@markdown <br><center>PEERFLIX-SERVER don't delete removed torrent task file. So you do this manually after remove torrent task.
!touch /content/downloads/errorRemove
!cd downloads/ && sudo rm -r *
print("All Deleted on /content/peerflix")

# <img src='https://upload.wikimedia.org/wikipedia/commons/c/c5/Deluge_icon.png' height="30" alt="Deluge" /> __Deluge__

In [0]:
#@markdown <h3>⬅️ Click Here to START server</h3>
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/c/c5/Deluge_icon.png' height="200" alt="netdata"/></center>
#@markdown <center><h3>Deluge<br />Deluge is a BitTorrent client written in Python.</h3></center><br>
#@markdown <center><h5>Auto upload to gdrive after task completed. if gdrive is mounted<br>Default output path /content/downloads/</h5></center><br>
import sys, shlex, subprocess, string, os, urllib.request, json, time; from IPython.display import HTML, clear_output

Version = '1.0.9'

#####################################
USE_FREE_TOKEN = True  # @param {type:"boolean"}
TOKEN = ""  # @param {type:"string"}
PORT = 8112

if not os.path.exists("/root/.ipython/ttmg.py"):
    shellCmd = "wget -qq https://raw.githubusercontent.com/biplobsd/" \
                "Google-Colab-CloudTorrent/master/res/ttmg.py \
                    -O /root/.ipython/ttmg.py"
    subprocess.run(shlex.split(shellCmd))


from ttmg import (
    updateCheck,
    runSh,
    nameport,
    installNgrok,
    findProcess,
    displayUrl,
    loadingAn,
)

def exx():
  !kill -9 -1 &

def ngrok_config(token, region, port, configPath="/root/.ngrok2/ngrok.yml"):
  data = """
  authtoken: {}
  region: {}
  update: false
  update_channel: stable
  web_addr: localhost:{}
  tunnels:
    deluge:
      addr: 8112
      proto: http
      inspect: false
  """.format(token, region, port).split('\n')
  try:
    os.mkdir('/root/.ngrok2/')
  except:
    pass
  open(configPath, 'w').close()
  with open(configPath, "a+") as configFile:
    for line in data:
      configFile.write(line + "\n")
  return True

def startWebUi(data, dport, nServer, region, btc, configPath):
    tokens, WEB_PORT = data
    from json import loads

    if tokens == "Invalid Token":
        print(tokens)
        exx()

    installNgrok()
    clear_output(wait=True)
    loadingAn(name="lds")
    print("Starting ngrok ...")
    ngrok_config(tokens, region, dport, configPath)
    shellCmd = f"ngrok start --config {configPath} --all &"
    runSh(shellCmd, shell=True)
    time.sleep(7)
    try:
        host = urllib.request.urlopen(f"http://localhost:{dport}/api/tunnels")
        host = json.loads(host.read())['tunnels']
        for h in host:
          if h['name'] == nServer:
            host = h['public_url'][8:]
            break
    except:
        print("ngrok Token is in used!. Try another token ...")
        time.sleep(2)
        return True
    
    data = {
        "url": f"http://{host}",
        "port": WEB_PORT,
        "token": tokens,
    }
    displayUrl(data, btc)
    return data

loadingAn()

# DELUGE SETUP
try:
  os.mkdir("/root/.config/deluge/")
  os.mkdir("/content/downloads/")
except FileExistsError:
  pass

if not findProcess("deluged"):
  runSh('sudo apt install -y deluged deluge-console deluge-webui')
  runSh(
        "wget -qq https://raw.githubusercontent.com/biplobsd/Google-Colab-CloudTorrent/master/res/deluge/core.conf \
            -O /root/.config/deluge/core.conf"
    )
  runSh(
        "wget -qq https://raw.githubusercontent.com/biplobsd/Google-Colab-CloudTorrent/master/res/deluge/web.conf \
            -O /root/.config/deluge/web.conf"
    )
  runSh('deluged &> /dev/null &', shell=True)
  runSh('deluge-web --fork', shell=True)
  runSh("""sed -i 's/if s.hexdigest() == config\["pwd_sha1"\]:/if True:/' /usr/lib/python2.7/dist-packages/deluge/ui/web/auth.py""")
  runSh("sed -i 's/onShow:function(){this.passwordField.focus(.*)}/onShow:function(){this.onLogin();}/' /usr/lib/python2.7/dist-packages/deluge/ui/web/js/deluge-all.js")
else:
  runSh('pkill deluged')
  runSh('pkill deluge-web')
  runSh('deluged &> /dev/null &', shell=True)
  runSh('deluge-web --fork', shell=True)

# START_SERVER
dport = 4041
nServer = 'deluge' 
configPath = "/root/.ngrok2/ngrok2.yml"
region = 'eu'
btc = 'b'

try:
  host = urllib.request.urlopen(f"http://localhost:{dport}/api/tunnels")
  host = json.loads(host.read())['tunnels']
  succ = False
  for h in host:
    if h['name'] == nServer:
      host = h['public_url'][8:]
      data = {
        "url": f"http://{host}",
        "port": PORT,
      }
      displayUrl(data, btc)
      succ = True
      break
  if not succ:
    raise Exception('Not found tunnels')
except:
  for run in range(10):
    clear_output()
    loadingAn(name='lds')
    try:
      dati = startWebUi(
          nameport(TOKEN, USE_FREE_TOKEN, PORT),
          dport,
          nServer,
          region,
          btc,
          configPath
          )
      if dati == True:
        continue
      urllib.request.urlopen(dati['url'])
      break
    except:
      clear_output()
      print(f"[{run}] Error: Ngrok not starting. Retrying ...")

# <img src='http://pic2.orsoon.com/2017/0719/20170719091439708.png' height="60" alt="Deluge" />___aria2___

In [0]:
#@markdown <h3>⬅️ Click Here to START server</h3>
#@markdown <br><center><img src='http://pic2.orsoon.com/2017/0719/20170719091439708.png' height="200" alt="netdata"/></center>
#@markdown <center><h3>aria2<br />The next generation download utility.</h3></center><br>
import sys
import shlex
import subprocess
import string
import os
import urllib.request
import json
import time
from IPython.display import HTML, clear_output

Version = '1.0.9'

#####################################
USE_FREE_TOKEN = True  # @param {type:"boolean"}
# Aria2 Service
Aria2_rpc = True  # @param {type:"boolean"}
Aria2ng_WEBUI = True  # @param {type:"boolean"}

TOKEN = ""  # @param {type:"string"}
PORT = 8221

if not os.path.exists("/root/.ipython/ttmg.py"):
    shellCmd = "wget -qq https://raw.githubusercontent.com/biplobsd/" \
                "Google-Colab-CloudTorrent/master/res/ttmg.py \
                    -O /root/.ipython/ttmg.py"
    subprocess.run(shlex.split(shellCmd))


from ttmg import (
    updateCheck,
    runSh,
    nameport,
    installNgrok,
    findProcess,
    displayUrl,
    loadingAn,
)

def exx():
  !kill -9 -1 &

def aria2Install():
  runSh('apt install -y aria2')

def ngrok_config(token, region, port, configPath="/root/.ngrok2/ngrok.yml"):
  data = """
  authtoken: {}
  region: {}
  update: false
  update_channel: stable
  web_addr: localhost:{}
  tunnels:
    Aria2_rpc:
      addr: 6800
      proto: http
      inspect: false
    Aria2ng:
      addr: 8221
      proto: http
      inspect: false
  """.format(token, region, port).split('\n')
  try:
    os.mkdir('/root/.ngrok2/')
  except:
    pass
  open(configPath, 'w').close()
  with open(configPath, "a+") as configFile:
    for line in data:
      configFile.write(line + "\n")
  return True

def startWebUi(data, dport, nServer, region, btc, configPath, displayB):
    tokens, WEB_PORT = data
    from json import loads

    if tokens == "Invalid Token":
        print(tokens)
        exx()

    installNgrok()
    clear_output(wait=True)
    loadingAn(name="lds")
    print("Starting ngrok ...")
    ngrok_config(tokens, region, dport, configPath)
    shellCmd = f"ngrok start --config {configPath} --all &"
    runSh(shellCmd, shell=True)
    time.sleep(7)
    try:
        host = urllib.request.urlopen(f"http://localhost:{dport}/api/tunnels")
        host = json.loads(host.read())['tunnels']
        for h in host:
          if h['name'] == nServer:
            host = h['public_url'][8:]
            break
    except:
        print("ngrok Token is in used!. Try another token ...")
        time.sleep(2)
        return True
    
    data = {
        "url": f"http://{host}",
        "port": WEB_PORT,
        "token": tokens,
    }
    if displayB:
      displayUrl(data, btc)
    return data

def ngrok_ServerStart(nServer, dport, region, btc, configPath, displayB=True):
  try:
    host = urllib.request.urlopen(f"http://localhost:{dport}/api/tunnels")
    host = json.loads(host.read())['tunnels']
    succ = False
    for h in host:
      if h['name'] == nServer:
        host = h['public_url'][8:]
        data = {
          "url": f"http://{host}",
          "port": PORT,
        }
        if displayB:
          displayUrl(data, btc)
        succ = True
        return data
    if not succ:
      raise Exception('Not found tunnels')
  except:
    for run in range(10):
      clear_output()
      loadingAn(name='lds')
      try:
        dati = startWebUi(
            nameport(TOKEN, USE_FREE_TOKEN, PORT),
            dport,
            nServer,
            region,
            btc,
            configPath,
            displayB
            )
        if dati == True:
          continue
        if not nServer == 'Aria2_rpc':
          urllib.request.urlopen(dati['url'])
        return dati
      except:
        clear_output()
        print(f"[{run}] Error: Ngrok not starting. Retrying ...")

loadingAn()

# Aria2ng SETUP
aria2Install()
try:
  os.mkdir("downloads/")
  os.mkdir("aria2ng/")
except FileExistsError:
  pass

# Downloading latest version of aria2ng
if not os.path.exists("/content/aria2ng/index.html"):
  BASE_URL = r"https://github.com/mayswind/AriaNg"
  shellCmd = f"""curl -s "{BASE_URL}/releases/latest" """ \
              r"""| sed -En 's/.+\/tag\/([.0-9A-Za-z]+)".+/\1/p'"""
  LATEST_TAG = subprocess.check_output(shellCmd, shell=True).decode('UTF-8')
  shellCmd = r'curl -L -# -C - ' \
              f'"{BASE_URL}/releases/download/{LATEST_TAG}/' \
              f'AriaNg-{LATEST_TAG}-AllInOne.zip" ' \
              r'-o /content/aria2ng/new.zip'
  runSh(shellCmd)
  shellCmd = "unzip -o /content/aria2ng/new.zip " \
              "-d /content/aria2ng/"
  runSh(shellCmd, output=True)
  runSh("rm -f /content/aria2ng/new.zip")
  # Enable insecure protocol
  shellCmd = r"""sed -Ei """ \
            r"""'s/"https"===[0-9A-Za-z]+\.protocol\(\)/ false/g' """ \
            r"""/content/aria2ng/index.html"""
  runSh(shellCmd)

# START_ARIA2NG_WEBUI
try:
  urllib.request.urlopen(f"http://localhost:{PORT}")
except:
  DIR_DL = "/content/downloads/"
  DIR_WEBUI = "/content/aria2ng/"
  if Aria2_rpc:
    runSh(
        f"aria2c --enable-rpc --rpc-listen-port=6800 -D -d {DIR_DL} &",
        shell=True
    )
  if Aria2ng_WEBUI:
    runSh(f"python3 -m http.server {PORT} &", shell=True, cd=DIR_WEBUI)

# START_SERVER
# Ngrok region 'us','eu','ap','au','sa','jp','in'
region = 'in'
if Aria2_rpc:
  data = ngrok_ServerStart(
      'Aria2_rpc',
      4041,
      region,
      'b',
      "/root/.ngrok2/ngrok2.yml", 
      displayB=False
      )
clear_output()
if Aria2ng_WEBUI:
  ngrok_ServerStart(
      'Aria2ng',
      4041,
      region,
      'b',
      "/root/.ngrok2/ngrok2.yml", 
      displayB=True
      )

if Aria2_rpc:
  Host = data['url'][7:]
  display(HTML("""<style>@import url('https://fonts.googleapis.com/css?family=Source+Code+Pro:200,900');  :root {   --text-color: hsla(210, 50%, 85%, 1);   --shadow-color: hsla(210, 40%, 52%, .4);   --btn-color: hsl(210, 80%, 42%);   --bg-color: #141218; }  * {   box-sizing: border-box; } button { position:relative; padding: 10px 20px;     border: none;   background: none;      font-family: "Source Code Pro";   font-weight: 900;font-size: 20px;     color: var(--text-color);      background-color: var(--btn-color);   box-shadow: var(--shadow-color) 2px 2px 22px;   border-radius: 4px;    z-index: 0;overflow: hidden; -webkit-user-select: text;-moz-user-select: text;-ms-user-select: text;user-select: text;}  button:focus {   outline-color: transparent;   box-shadow: var(--btn-color) 2px 2px 22px; }  .right::after, button::after {   content: var(--content);   display: block;   position: absolute;   white-space: nowrap;   padding: 40px 40px;   pointer-events:none; }  button::after{   font-weight: 200;   top: -30px;   left: -20px; }   .right, .left {   position: absolute;   width: 100%;   height: 100%;   top: 0; } .right {   left: 66%; } .left {   right: 66%; } .right::after {   top: -30px;   left: calc(-66% - 20px);      background-color: var(--bg-color);   color:transparent;   transition: transform .4s ease-out;   transform: translate(0, -90%) rotate(0deg) }  button:hover .right::after {   transform: translate(0, -47%) rotate(0deg) }  button .right:hover::after {   transform: translate(0, -50%) rotate(-7deg) }  button .left:hover ~ .right::after {   transform: translate(0, -50%) rotate(7deg) }  /* bubbles */ button::before {   content: '';   pointer-events: none;   opacity: .6;   background:     radial-gradient(circle at 20% 35%,  transparent 0,  transparent 2px, var(--text-color) 3px, var(--text-color) 4px, transparent 4px),     radial-gradient(circle at 75% 44%, transparent 0,  transparent 2px, var(--text-color) 3px, var(--text-color) 4px, transparent 4px),     radial-gradient(circle at 46% 52%, transparent 0, transparent 4px, var(--text-color) 5px, var(--text-color) 6px, transparent 6px);    width: 100%;   height: 300%;   top: 0;   left: 0;   position: absolute;   animation: bubbles 5s linear infinite both; }  @keyframes bubbles {   from {     transform: translate();   }   to {     transform: translate(0, -66.666%);   } }.zui-table {    border: solid 1px #DDEEEE;    border-collapse: collapse;    border-spacing: 0;    font: normal 13px;}.zui-table thead th {    background-color: #DDEFEF;    border: solid 1px #DDEEEE;    color: #0000009e;    padding: 10px;    text-align: left;}.zui-table tbody td {border: solid 1px #effff97a;color: #ffffffd1;    padding: 10px;}</style><center><button><table class="zui-table blueBG"><p>Aria2 rpc config<p><thead>        <tr>            <th>Protocol</th>            <th>Host</th>            <th>Port</th>        </tr>    </thead>    <tbody>        <tr>            <td>WebSocket</td><td>"""+Host+"""</td><td>80</td></tr></tbody></table><a target="_blank" style="text-decoration: none;color: hsla(210, 50%, 85%, 1);font-size: 10px;" href="http://bit.ly/2qoOw0a">NB. How to setup this's config. [Click ME]</a></button><center>"""))


# <img src='https://i.imgur.com/fC5xi1Q.png' height="45" alt="wget"/> <font size=1px >___Direct link downloader [beta]___</font>

___Auto share link supported : Disk.yandex.com, Gdrive___

In [0]:
URL = "" #@param {type:"string"}
#@markdown <center><h5>Default output path /content/downloads/</h5></center>
OUTPUT_PATH = "" #@param {type:"string"}

import pathlib
import shutil
import hashlib
import requests
from urllib.parse import urlparse
from os import path, mkdir
if not path.exists("/root/.ipython/ttmg.py"): 
    from subprocess import run
    from shlex import split

    shellCmd = "wget -qq https://raw.githubusercontent.com/biplobsd/Google-Colab-CloudTorrent/master/res/ttmg.py \
                    -O /root/.ipython/ttmg.py"
    run(split(shellCmd))
from ttmg import updateCheck, runSh


def aria2Install():
  runSh('apt install -y aria2')

def GetDirLink(link):
    link = urlparse(URL)
    _,id = link.path.split("d/")
    id,_ =  id.split("/edit")
    dtr = "https://drive.google.com/uc?export=download&id="
    directlink = dtr + id
    return directlink

def istmd(URL): 
  link = urlparse(URL)

  #Gdrive
  if link.netloc == "drive.google.com":
    print('URL {}'.format(URL))
    if (link.path.find('/file/d/') != -1):
      URL = GetDirLink(URL)
    runSh(f'curl -JLO {URL}',output=True, cd=OUTPUT_PATH)
    
    return False
    
  #YandexDisk
  if link.netloc == "yadi.sk":

    API_ENDPOINT = 'https://cloud-api.yandex.net/v1/disk/public/resources/?public_key={}&path=/{}&offset={}'
    dry = False


    def md5sum(file_path):
        md5 = hashlib.md5()
        with open(file_path, 'rb') as f:
            for chunk in iter(lambda: f.read(128 * md5.block_size), b''):
                md5.update(chunk)
        return md5.hexdigest()


    def check_and_download_file(target_path, url, size, checksum):
        if path.isfile(target_path):
            if size == path.getsize(target_path):
                if checksum == md5sum(target_path):
                    print('URL {}'.format(url))
                    print('skipping correct {}'.format(target_path))
                    return
        if not dry:
            print('URL {}'.format(url))
            print('downloading {}'.format(target_path))
            runSh(f'aria2c -x8 -d {OUTPUT_PATH} {url}', output=True)
            # r = requests.get(url, stream=True)
            # with open(target_path, 'wb') as f:
            #     shutil.copyfileobj(r.raw, f)


    def download_path(target_path, public_key, source_path, offset=0):
        print('getting "{}" at offset {}'.format(source_path, offset))
        current_path = path.join(target_path, source_path)
        pathlib.Path(current_path).mkdir(parents=True, exist_ok=True)
        jsn = requests.get(API_ENDPOINT.format(public_key, source_path, offset)).json()
        def try_as_file(j):
            if 'file' in j:
                file_save_path = path.join(current_path, j['name'])
                check_and_download_file(file_save_path, j['file'], j['size'], j['md5'])
                return True
            return False

        # first try to treat the actual json as a single file description
        if try_as_file(jsn):
            return

        # otherwise treat it as a directory
        emb = jsn['_embedded']
        items = emb['items']
        for i in items:
            # each item can be a file...
            if try_as_file(i):
                continue
            # ... or a directory
            else:
                subdir_path = path.join(source_path, i['name'])
                download_path(target_path, public_key, subdir_path)

        # check if current directory has more items
        last = offset + emb['limit']
        if last < emb['total']:
            download_path(target_path, public_key, source_path, last)
    download_path(OUTPUT_PATH, URL, '')
    return False  
  return URL

if not OUTPUT_PATH:
  OUTPUT_PATH = "/content/downloads/"
  
if not URL == "":
  aria2Install()
  try:
    mkdir("downloads")
  except FileExistsError:
    pass
  url = istmd(URL)
  if url != False:
    print('URL {}'.format(URL))
    runSh(f'aria2c -x8 -d {OUTPUT_PATH} {url}', output=True)
else:
  print("Please input url")



#<img src='http://pngimages.net/sites/default/files/zip-file-png-image-15464.png' height="30" alt="netdata"> __Extracte Zip/Tar/Rar__

<font size=2px><a href="https://github.com/geart891/RLabClone/">___Credit : @geart891___</a></font>

In [0]:
# ============================= FORM ============================= #
# @markdown #### ⬅️ Extract Files
MODE = "UNZIP"  # @param ["UNZIP", "UNTAR", "UNRAR"]
PATH_TO_FILE = ""  # @param {type:"string"}
# ================================================================ #

from os import path as _p

if not _p.exists("/root/.ipython/rlab_utils.py"):
    from shlex import split as _spl
    from subprocess import run  # nosec

    shellCmd = "wget -qq https://raw.githubusercontent.com/biplobsd/RLabClone/master/res/rlab_utils.py \
                    -O /root/.ipython/rlab_utils.py"
    run(_spl(shellCmd))  # nosec

from rlab_utils import checkAvailable, runSh


def extractFiles():
    extractPath = "/content/extract"
    if not checkAvailable("/content/extract"):
        runSh("mkdir -p -m 777 /content/extract")
    if MODE == "UNZIP":
        runSh(f'unzip "{PATH_TO_FILE}" -d "{extractPath}"', output=True)
    elif MODE == "UNRAR":
        runSh(f'unrar x "{PATH_TO_FILE}" "{extractPath}"', output=True)
    else:
        runSh(f'tar -C "{extractPath}" -xvf "{PATH_TO_FILE}"', output=True)


extractFiles()


# <img src='https://geart891.github.io/RLabClone/img/title_rclonelab.svg' height="45" alt="RcloneLab"/><font size=1px>___Use it for upload data to Gdrive___</font>

<font size=2px><a href="https://github.com/geart891/RLabClone/">___Credit : @geart891___</a></font>

In [0]:
# ============================= FORM ============================= #
# @markdown #### ⬅️ Execute rClone
# @markdown ##### Quick move (check and leave all boxs):
# @markdown #####{
SimpleTorrent = False # @param{type: "boolean"}
Peerflix = False # @param{type: "boolean"}
# @markdown ##### }
Mode = "Move"  # @param ["Move", "Copy", "Sync", "Verify", "Dedupe", "Clean Empty Dirs", "Empty Trash"]
Compare = "Size & Checksum"  # @param ["Size & Checksum", "Size & Mod-Time", "Mod-Time", "Size", "Checksum"]
Source = ""  # @param {type:"string"}
Destination = ""  # @param {type:"string"}
Extra_Arguments = ""  # @param {type:"string"}
COPY_SHARED_FILES = False  # @param{type: "boolean"}
TRANSFERS, CHECKERS = 20, 20
THROTTLE_TPS = True
BRIDGE_TRANSFER = False  # @param{type: "boolean"}
FAST_LIST = False  # @param{type: "boolean"}
OPTIMIZE_GDRIVE = True
SIMPLE_LOG = True
RECORD_LOGFILE = False  # @param{type: "boolean"}
SKIP_NEWER_FILE = False
SKIP_EXISTED = False
SKIP_UPDATE_MODTIME = False
ONE_FILE_SYSTEM = False
LOG_LEVEL = "DEBUG"
SYNC_MODE = "Delete after transfering"
SYNC_TRACK_RENAME = True
DEDUPE_MODE = "Largest"
USE_TRASH = True
DRY_RUN = False  # @param{type: "boolean"}
# ================================================================ #
if SimpleTorrent and Source == "" and Destination == "" and Peerflix == False:
  Mode = "Move"
  Source = "/content/downloads"
  Destination = "/content/drive/My Drive" 
elif Peerflix and Source == "" and Destination == "" and SimpleTorrent == False:
  Mode = "Move"
  Source = "/content/peerflix"
  Destination = "/content/drive/My Drive" 

from os import path as _p

if not _p.exists("/root/.ipython/rlab_utils.py"):
    from shlex import split as _spl
    from subprocess import run  # nosec

    shellCmd = "wget -qq https://raw.githubusercontent.com/biplobsd/RLabClone/master/res/rlab_utils.py \
                    -O /root/.ipython/rlab_utils.py"
    run(_spl(shellCmd))  # nosec

from datetime import datetime as _dt
from rlab_utils import (
    displayOutput,
    checkAvailable,
    runSh,
    prepareSession,
    PATH_RClone_Config,
    accessSettingFile,
    memGiB,
)


def populateActionArg():
    if Mode == "Copy":
        actionArg = "copy"
    elif Mode == "Sync":
        actionArg = "sync"
    elif Mode == "Verify":
        actionArg = "check"
    elif Mode == "Dedupe":
        actionArg = "dedupe largest"
    elif Mode == "Clean Empty Dirs":
        actionArg = "rmdirs"
    elif Mode == "Empty Trash":
        actionArg = "delete"
    else:
        actionArg = "move"

    return actionArg


def populateCompareArg():
    if Compare == "Mod-Time":
        compareArg = "--ignore-size"
    elif Compare == "Size":
        compareArg = "--size-only"
    elif Compare == "Checksum":
        compareArg = "-c --ignore-size"
    else:
        compareArg = "-c"

    return compareArg


def populateOptimizeGDriveArg():
    return (
        "--buffer-size 256M \
            --drive-chunk-size 256M \
                --drive-upload-cutoff 256M \
                    --drive-acknowledge-abuse \
                        --drive-keep-revision-forever"
        if OPTIMIZE_GDRIVE
        else "--buffer-size 128M"
    )


def populateGDriveCopyArg():
    if BRIDGE_TRANSFER and memGiB() < 13:
        global TRANSFERS, CHECKERS
        TRANSFERS, CHECKERS = 10, 80
    else:
        pass
    return "--disable copy" if BRIDGE_TRANSFER else "--drive-server-side-across-configs"


def populateStatsArg():
    statsArg = "--stats-one-line --stats=5s" if SIMPLE_LOG else "--stats=5s -P"
    if LOG_LEVEL != "OFF":
        statsArg += " -v" if SIMPLE_LOG else "-vv"
    elif LOG_LEVEL == "INFO":
        statsArg += " --log-level INFO"
    elif LOG_LEVEL == "ERROR":
        statsArg += " --log-level ERROR"
    else:
        statsArg += " --log-level DEBUG"
    return statsArg


def populateSyncModeArg():
    if Mode != "Sync":
        return ""
    elif SYNC_MODE == "Delete before transfering":
        syncModeArg = "--delete-before"
    elif SYNC_MODE == "Delete after transfering":
        syncModeArg = "--delete-after"
    else:
        syncModeArg = "--delete-during"
    if SYNC_TRACK_RENAME:
        syncModeArg += " --track-renames"
    return syncModeArg


def populateDedupeModeArg():
    if DEDUPE_MODE == "Interactive":
        dedupeModeArg = "--dedupe-mode interactive"
    elif DEDUPE_MODE == "Skip":
        dedupeModeArg = "--dedupe-mode skip"
    elif DEDUPE_MODE == "First":
        dedupeModeArg = "--dedupe-mode first"
    elif DEDUPE_MODE == "Newest":
        dedupeModeArg = "--dedupe-mode newest"
    elif DEDUPE_MODE == "Oldest":
        dedupeModeArg = "--dedupe-mode oldest"
    elif DEDUPE_MODE == "Rename":
        dedupeModeArg = "--dedupe-mode rename"
    else:
        dedupeModeArg = "--dedupe-mode largest"

    return dedupeModeArg


def generateCmd():
    sharedFilesArgs = (
        "--drive-shared-with-me --files-from /content/upload.txt --no-traverse"
        if COPY_SHARED_FILES
        else ""
    )

    logFileArg = f"--log-file /content/rclone_log.txt -vv -P"

    args = [
        "rclone",
        f"--config {PATH_RClone_Config}/rclone.conf",
        '--user-agent "Mozilla"',
        populateActionArg(),
        f'"{Source}"',
        f'"{Destination}"' if Mode in ("Move", "Copy", "Sync") else "",
        f"--transfers {str(TRANSFERS)}",
        f"--checkers {str(CHECKERS)}",
    ]

    if Mode == "Verify":
        args.append("--one-way")
    elif Mode == "Empty Trash":
        args.append("--drive-trashed-only --drive-use-trash false")
    else:
        args.extend(
            [
                populateGDriveCopyArg(),
                populateSyncModeArg(),
                populateCompareArg(),
                populateOptimizeGDriveArg(),
                "-u" if SKIP_NEWER_FILE else "",
                "--ignore-existing" if SKIP_EXISTED else "",
                "--no-update-modtime" if SKIP_UPDATE_MODTIME else "",
                "--one-file-system" if ONE_FILE_SYSTEM else "",
                "--tpslimit 95 --tpslimit-burst 40" if THROTTLE_TPS else "",
                "--fast-list" if FAST_LIST else "",
                "--delete-empty-src-dirs" if Mode == "Move" else "",
            ]
        )
    args.extend(
        [
            "-n" if DRY_RUN else "",
            populateStatsArg() if not RECORD_LOGFILE else logFileArg,
            sharedFilesArgs,
            Extra_Arguments,
        ]
    )

    return args


def executeRclone():
    prepareSession()
    if Source.strip() == "":
        displayOutput("❌ The Source field is empty.")
        return
    if checkAvailable("/content/rclone_log.txt"):
        if not checkAvailable("/content/logfiles"):
            runSh("mkdir -p -m 666 /content/logfiles")
        job = accessSettingFile("job.txt")
        runSh(
            f'mv /content/rclone_log.txt /content/logfiles/{job["title"]}_{job["status"]}_logfile.txt'
        )

    onGoingJob = {
        "title": f'{Mode}_{Source}_{Destination}_{_dt.now().strftime("%a-%H-%M-%S")}',
        "status": "ongoing",
    }
    accessSettingFile("job.txt", onGoingJob)

    cmd = " ".join(generateCmd())
    runSh(cmd, output=True)  # nosec
    displayOutput(Mode, "success")

    onGoingJob["status"] = "finished"
    accessSettingFile("job.txt", onGoingJob)


executeRclone()


#![alt text](https://mega.nz/favicon.ico?v=3) __TRANSFERE to MEGA__





In [0]:
# @markdown <center><h2>INPUT YOUR MEGA ID</h2></center><br>
from functools import wraps
import errno
import os
import signal
import subprocess
import shlex


class TimeoutError(Exception):
    pass


def timeout(seconds=10, error_message=os.strerror(errno.ETIME)):
    def decorator(func):
        def _handle_timeout(signum, frame):
            raise TimeoutError(error_message)

        def wrapper(*args, **kwargs):
            signal.signal(signal.SIGALRM, _handle_timeout)
            signal.alarm(seconds)
            try:
                result = func(*args, **kwargs)
            finally:
                signal.alarm(0)
            return result

        return wraps(func)(wrapper)

    return decorator


if not os.path.exists("/root/.ipython/ttmg.py"):
    from subprocess import run
    from shlex import split

    shellCmd = "wget -qq https://raw.githubusercontent.com/biplobsd/Google-Colab-CloudTorrent/master/res/ttmg.py \
                    -O /root/.ipython/ttmg.py"
    run(split(shellCmd))
from ttmg import runSh

@timeout(10)
def runShT(args):
    return runSh(args, output=True)


# MEGAcmd installing
if not os.path.exists("/usr/bin/mega-cmd"):
    print("Installing MEGA ...")
    runSh('sudo apt-get -y update')
    runSh('sudo apt-get -y install libmms0 libc-ares2 libc6 libcrypto++6 libgcc1 libmediainfo0v5 libpcre3 libpcrecpp0v5 libssl1.1 libstdc++6 libzen0v5 zlib1g apt-transport-https')
    runSh('sudo curl -sL -o /var/cache/apt/archives/MEGAcmd.deb https://mega.nz/linux/MEGAsync/Debian_9.0/amd64/megacmd-Debian_9.0_amd64.deb', output=True)
    runSh('sudo dpkg -i /var/cache/apt/archives/MEGAcmd.deb', output=True)

    print("MEGA is installed.")
else:
    !pkill mega-cmd

# INPUT YOUR MEGA ID

USERNAME = ""  # @param {type:"string"}
PASSWORD = ""  # @param {type:"string"}
if not (USERNAME == "" or PASSWORD == ""):
    try:
        runShT(f"mega-login {USERNAME} {PASSWORD}")
    except TimeoutError:
        runSh('mega-whoami', output=True)
else:
    print("Please Input your Mega IDs.")


In [0]:
# @markdown ##### Quick move (check and leave all boxs):
# @markdown #####{
import time
import subprocess
import contextlib
from IPython.display import clear_output
Simple_torrent = False  # @param{type: "boolean"}
Peerflix = False  # @param{type: "boolean"}
# @markdown ##### }
# @markdown <br><center><img src='https://www.impactcomputing.co.uk/uploads/files/services/managed-office-365.svg' height="100" alt="netdata"/></center>
# @markdown <center><h2>TRANSFERE NOW</h2></center><br>
# TRANSFERE NOW

PATH_TO_FILE = ""  # @param {type:"string"}
# Unix, Windows and old Macintosh end-of-line
newlines = ['\n', '\r\n', '\r']


def unbuffered(proc, stream='stdout'):
    stream = getattr(proc, stream)
    with contextlib.closing(stream):
        while True:
            out = []
            last = stream.read(1)
            # Don't loop forever
            if last == '' and proc.poll() is not None:
                break
            while last not in newlines:
                # Don't loop forever
                if last == '' and proc.poll() is not None:
                    break
                out.append(last)
                last = stream.read(1)
            out = ''.join(out)
            yield out


def transfare():
    cmd = ""
    if Simple_torrent:
        cmd = ['mega-put', 'downloads', '/colab']
    elif Peerflix:
        cmd = ['mega-put', 'peerflix', '/colab']
    else:
        cmd = ['mega-put', PATH_TO_FILE, '/colab']
    proc = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        # Make all end-of-lines '\n'
        universal_newlines=True,
    )
    for line in unbuffered(proc):
        clear_output(wait=True)
        print(line)

try:
    transfare()
except FileNotFoundError:
    print("Login your account!")
